<a href="https://colab.research.google.com/github/Charmaineem/Image-Caption-Generator/blob/main/Image_caption_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio #used for creating the demo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.2/305.2 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 996.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.0 MB/s 

In [2]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.4 MB/s eta 0:00:00


In [3]:
!pip install huggingface_hub

from huggingface_hub import notebook_login

notebook_login()

In [4]:
import gradio as gr
import requests
import matplotlib.pyplot as plt
import io
from PIL import Image
from matplotlib.patches import Rectangle


In [49]:
#Load model directly
from transformers import AutoProcessor, BlipForConditionalGeneration, pipeline

# Loading the BLIP model directly which generates the caption
processor = AutoProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

#Using transformers to load DETR model for object detection
#This model adds a bounding box and label to detected objects
object_detector = pipeline("object-detection", model="facebook/detr-resnet-50")

Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Could not find image processor class in the image processor config or the model conf

In [51]:
#generates the caption for uploaded image
def caption_generator(input_img):
  inputs = processor(input_img, return_tensors="pt")
  out = model.generate(**inputs, max_new_tokens=500)
  caption = processor.decode(out[0], skip_special_tokens=True)
  return caption

In [63]:
#function to filter the generated caption checking whether human, cats and/or dogs are present using the labels from the object detection
#this is the method used in this project
def filter_caption(object_detection_results):
  labels = [result['label'] for result in object_detection_results]
  keywords = ["dog","dogs", "cat","cats", "human","humans","man", "men","woman","women","child","children","adult","adults","person"]
  return True if any(keyword in labels for keyword in keywords) else False

In [52]:
#function to filter the generated caption checking whether human, cats and/or dogs are present using the generated caption
#initial method considered
def filter(caption):
  #If any of these keywords are present, True is returned
  keywords = ["dog","dogs", "cat","cats", "human","humans","man", "men","woman","women","child","children","adult","adults","person"]
  caption = caption.lower()
  return True if any(keyword in caption for keyword in keywords) else False

In [53]:
#function to create the bounding box and label
#takes an image and list of results as inputs
def create_image_bbx_w_label(image, results):

  # Set up the plot
  fig, ax = plt.subplots(figsize=(12, 8))
  ax.imshow(image)

  # Plot the bounding boxes and labels
  for res in results:
      box = res['box']
      width = box['xmax'] - box['xmin']
      height = box['ymax'] - box['ymin']

      rect = Rectangle((box['xmin'], box['ymin']), width, height, linewidth=1, edgecolor='r', facecolor='none')
      ax.add_patch(rect)

      # Position the label above the rectangle
      label_position = (box['xmin'], box['ymin'] - 10)

      # Display the label and score
      label_text = f"{res['label']}: {res['score']:.2f}"
      ax.text(*label_position, label_text, color='white', fontsize=8, bbox=dict(facecolor='red', alpha=0.5))

  ax.axis('off')

  fname = './img.png'
  plt.savefig(fname, format='png', bbox_inches='tight', pad_inches=0)

  plt.close(fig)
  # Load this buffer into a PIL Image
  pil_img = Image.open(fname)

  # Return the PIL Image object
  return pil_img


In [65]:
def image_caption_generator(input_image):
  #detecting onjects in image
  object_detection_results = object_detector(input_image)
  annotated_img = create_image_bbx_w_label(input_image, object_detection_results)

  #Generating caption of input image
  caption = caption_generator(input_image)
  #Filtering the captions for specific case (humans and/or cats/dogs)
  #filtered_caption = filter(caption) uncomment this if you want to filter using the generated caption
  filtered_caption = filter_caption(object_detection_results) #uses the generated labels from object detection to filter the captions
  if filtered_caption:
    return caption, annotated_img
  else:
    return "There are no humans or pets in this image!", annotated_img

In [66]:
demo = gr.Interface(fn = image_caption_generator,
                    inputs=[gr.Image(label="Upload image", type="pil")],
                    outputs=[gr.Textbox(label="Caption"), 'image'],
                    title="CaptionPlus - Image Caption Generator",
                    description="Captioning images of humans, cats and/or dogs with object detection",
                    allow_flagging="never",
                    examples=['/content/OIP (1).jpg','/content/car.jpg','/content/Example.jpg'])

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://b7e33b893ef4003c4e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
